In [1]:
import pandas as pd
import numpy as np


In [2]:
counts_sex_age_state = pd.read_csv('cdc_death_counts_by_sex_age_state.csv')
counts_sex_age_state.head()

,Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
0,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,All Ages,974282.0,7520550.0,855731.0,504882.0,10684.0,1334231.0,NaN
1,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,Under 1 year,242.0,41777.0,495.0,28.0,28.0,736.0,NaN
2,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,0-17 years,933.0,74176.0,1460.0,252.0,204.0,2342.0,NaN
3,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,1-4 years,113.0,7791.0,293.0,29.0,70.0,447.0,NaN
4,03/23/2022,01/01/2020,03/19/2022,By Total,NaN,NaN,United States,All Sexes,5-14 years,296.0,12438.0,424.0,96.0,82.0,705.0,NaN


In [3]:
from sklearn.preprocessing import OneHotEncoder
def onehotencoder(data, column_name): 
    """
    One hot encodes a categorical feature. 
    Input:
      dataframe
      column_name with column_name to be one-hot-encoded 
    Output:
      A dataframe, with the one hot encoded features as columns.
    """
    column_data = data[[column_name]]
    ohe = OneHotEncoder()
    enc = ohe.fit_transform(column_data)
    enc_array = enc.toarray()
    one_hot_encoded_df = pd.DataFrame(data=enc_array, columns=ohe.get_feature_names_out\
                                      (input_features=[column_name]), index=data.index)
    data = data.drop(columns=[column_name])
    data = pd.concat([data, one_hot_encoded_df], axis=1)

    return data


jan - winter
feb - winter
march - spring
april- spring
may - spring
june - summer
july- summer
aug -summer
sept -fall
oct -fall
nov - fall
dec - winter

radomly pick 3 csvs from each month and group by season 

In [6]:
import os
import random
import pandas as pd

# Define the seasons
months = {
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
    '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
    '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
}

# Set the directory path where the CSVs are stored
directory = '../datasets/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us'

selected_csvs_by_month = {month: [] for month in months.keys()}

# List all files in the directory
files = os.listdir(directory)

# Iterate through files and group them by month
for file in files:
    for month in months.keys():
        if month in file.lower() and file.endswith('.csv'):
            selected_csvs_by_month[month].append(file)

# Load the selected CSVs into dataframes
dataframes_by_month = {month: [] for month in months.values()}

for month, csvs in selected_csvs_by_month.items():
    for csv in csvs:
        # Construct the file path
        file_path = os.path.join(directory, csv)

        # Load CSV into a dataframe
        df = pd.read_csv(file_path)

        # Append dataframe to the corresponding month
        dataframes_by_month[months[month]].append(df)

# Debugging statement
print(f'Number of dataframes for Jul: {len(dataframes_by_month["Jul"])}')

# if len(dataframes_by_month['Oct']) > 0:
#     dataframes_by_month['Oct'][0]
# else:
#     print("No dataframes available for 'fall' season.")

# dataframes_by_month['Jun'][10]
drop_states = ['American Samoa', 'Diamond Princess', 'Northern Mariana Islands'\
               , 'Puerto Rico', 'Virgin Islands', 'Grand Princess']
for month, dataframes in dataframes_by_month.items():
    modified_dataframes = []
    for df in dataframes:
        modified_df = df[~df['Province_State'].isin(drop_states)]
        modified_dataframes.append(modified_df)
    dataframes_by_month[month] = modified_dataframes
# dataframes_by_month['Oct'][11]

Number of dataframes for Jul: 125


In [7]:
data = {month: [df.drop(columns=['Last_Update', 'Country_Region', 'UID', 'ISO3', 'Lat', 'Long_',\
                                 'Recovered', 'Active', 'People_Hospitalized',\
                                   'FIPS', 'Hospitalization_Rate'])\
                for df in dataframes] for month, dataframes in dataframes_by_month.items()}


data = {month: [df.assign(Month=month) for df in dataframes] 
        for month, dataframes in data.items()}

data['Oct'][1]

,Province_State,Confirmed,Deaths,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Date,People_Tested,Mortality_Rate,Month
0,Alabama,183831,2866,3749.216071,NaN,NaN,NaN,2020-10-24,2195615.0,1.559041,Oct
1,Alaska,12542,68,1714.453656,NaN,NaN,75558.714775,2020-10-24,552746.0,0.542178,Oct
3,Arizona,236772,5869,3252.935923,NaN,NaN,23267.136227,2020-10-24,2667371.0,2.478756,Oct
4,Arkansas,105318,1797,3489.888674,NaN,NaN,42415.544548,2020-10-24,1280018.0,1.706261,Oct
5,California,910428,17281,2304.236337,NaN,NaN,44566.409235,2020-10-24,17609179.0,1.899380,Oct
6,Colorado,106826,2394,1621.883691,NaN,NaN,31090.225355,2020-10-24,1790404.0,2.374732,Oct
7,Connecticut,66052,4577,1852.641877,NaN,NaN,59342.880391,2020-10-24,2115744.0,6.929389,Oct
8,Delaware,23847,709,2448.950670,551046.0,NaN,0.000006,2020-10-24,337276.0,2.973120,Oct
10,District of Columbia,16706,642,2367.130524,NaN,NaN,69649.833014,2020-10-24,491553.0,3.842931,Oct
11,Florida,776251,16417,3614.212242,NaN,NaN,52450.218568,2020-10-24,11265120.0,2.114909,Oct


In [5]:
# data = {month: [onehotencoder(df, 'Province_State') for df in dataframes] \
#                     for month, dataframes in data.items()}
# data['Oct'][10]

In [6]:
# month_df_encoded = {month: [onehotencoder(df, 'Month') for df in dataframes] \
#                     for month, dataframes in month_df.items()}

# month_str = [df['Last_Update'].astype('str')\
#              .str.split("-")[1] for dfs in month_df.values() for df in dfs]
# len(month_str)
# len(month_df['Oct'][1])
# month_df_with_month_column = {
#     month: [df.assign(Month=month_value) for df, month_value in zip(dfs, month_str * len(dfs))]
#     for month, dfs in month_df.items()
# }
# data.items()

In [8]:
concats = [pd.concat(df, axis=0) for df in data.values()]

all_data = pd.concat(concats, axis=0)


In [9]:
all_data = onehotencoder(all_data, 'Month')
all_data = onehotencoder(all_data, 'Province_State')
all_data.columns.values


/home/sam/miniconda3/envs/data_200s/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sam/miniconda3/envs/data_200s/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/sam/miniconda3/envs/data_200s/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sam/miniconda3/envs/data_200s/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstan

array(['Confirmed', 'Deaths', 'Incident_Rate', 'Total_Test_Results',
       'Case_Fatality_Ratio', 'Testing_Rate', 'Date', 'People_Tested',
       'Mortality_Rate', 'Month_Apr', 'Month_Aug', 'Month_Dec',
       'Month_Feb', 'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar',
       'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep',
       'Province_State_Alabama', 'Province_State_Alaska',
       'Province_State_Arizona', 'Province_State_Arkansas',
       'Province_State_California', 'Province_State_Colorado',
       'Province_State_Connecticut', 'Province_State_Delaware',
       'Province_State_District of Columbia', 'Province_State_Florida',
       'Province_State_Georgia', 'Province_State_Guam',
       'Province_State_Hawaii', 'Province_State_Idaho',
       'Province_State_Illinois', 'Province_State_Indiana',
       'Province_State_Iowa', 'Province_State_Kansas',
       'Province_State_Kentucky', 'Province_State_Louisiana',
       'Province_State_Maine', 'Province_State_Maryland',
 

In [10]:
print(all_data.dtypes)
all_data = all_data.astype(float)
nan_check = all_data.isna().any()

# Display columns with NaN values
columns_with_nan = nan_check[nan_check].index
print("Columns with NaN values:", columns_with_nan)
print(all_data.isna().sum())

Confirmed                         int64
Deaths                            int64
Incident_Rate                   float64
Total_Test_Results              float64
Case_Fatality_Ratio             float64
                                 ...   
Province_State_Virginia         float64
Province_State_Washington       float64
Province_State_West Virginia    float64
Province_State_Wisconsin        float64
Province_State_Wyoming          float64
Length: 74, dtype: object


ValueError: could not convert string to float: '2022-01-03'

In [11]:

# special_representations = ['NA', 'N/A', 'null']
# all_data.replace(special_representations, pd.NA, inplace=True)

# print(X['Deaths'].dtype)
import copy
X = copy.deepcopy(all_data).astype(float)

# X.dropna(subset=[col for col in X.columns if np.nan in X[col].values], inplace=True)
X = X.dropna(axis=1)
X
print(X.isna().sum())
target = X['Deaths']

# X_vals.drop(columns=['Deaths'], inplace=True)
X.shape

ValueError: could not convert string to float: '2022-01-03'

In [73]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42) 


In [75]:
model.fit(X_train, y_train) 

LinearRegression()

In [77]:
y_pred = model.predict(X_test)
y_pred

array([12514.,  7134.,  2038., ...,  3754.,  4084.,  9556.])

In [ ]:
import matplotlib.pyplot as plt

# Access the dataframe
df = dataframes_by_month['Oct'][0]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(df['Province_State'], df['Deaths'], color='orange')
plt.xlabel('Province_State')
plt.ylabel('Deaths')
plt.title('COVID-19 Deaths by Province_State (Fall Season)')
plt.xticks(rotation=90)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Access the dataframe
df = dataframes_by_season['summer'][0]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(df['Province_State'], df['Deaths'], color='pink')
plt.xlabel('Province_State')
plt.ylabel('Deaths')
plt.title('COVID-19 Deaths by Province_State (Summer Season)')
plt.xticks(rotation=90)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Access the dataframe
df = dataframes_by_season['winter'][0]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(df['Province_State'], df['Deaths'], color='lightblue')
plt.xlabel('Province_State')
plt.ylabel('Deaths')
plt.title('COVID-19 Deaths by Province_State (Winter Season)')
plt.xticks(rotation=90)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Access the dataframe
df = dataframes_by_season['spring'][0]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(df['Province_State'], df['Deaths'], color='lightgreen')
plt.xlabel('Province_State')
plt.ylabel('Deaths')
plt.title('COVID-19 Deaths by Province_State (Spring Season)')
plt.xticks(rotation=90)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import numpy as np

# Access the dataframes
fall_df = dataframes_by_season['fall'][0]
summer_df = dataframes_by_season['summer'][0]
winter_df = dataframes_by_season['winter'][0]
spring_df = dataframes_by_season['spring'][0]

# Set up the figure
plt.figure(figsize=(12, 8))

# Define the positions of the bars
x = np.arange(len(fall_df['Province_State']))  # the label locations
width = 0.2  # the width of the bars

# Plotting data
plt.bar(x - 0.3, fall_df['Deaths'], width, color='orange', label='Fall')
plt.bar(x - 0.1, summer_df['Deaths'], width, color='blue', label='Summer')
plt.bar(x + 0.1, winter_df['Deaths'], width, color='gray', label='Winter')
plt.bar(x + 0.3, spring_df['Deaths'], width, color='green', label='Spring')

# Add labels and title
plt.xlabel('Province_State')
plt.ylabel('Deaths')
plt.title('COVID-19 Deaths by Province_State (All Seasons)')
plt.xticks(x, fall_df['Province_State'], rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
# Access the dataframes
fall_df = dataframes_by_season['fall'][0]
summer_df = dataframes_by_season['summer'][0]
winter_df = dataframes_by_season['winter'][0]
spring_df = dataframes_by_season['spring'][0]

# Set up the figure
plt.figure(figsize=(12, 8))

# Define the positions of the bars
x = np.arange(len(fall_df['Province_State']))  # the label locations
width = 0.2  # the width of the bars

# Plotting data
plt.bar(x - 0.3, fall_df['Incident_Rate'], width, color='orange', label='Fall')
plt.bar(x - 0.1, summer_df['Incident_Rate'], width, color='blue', label='Summer')
plt.bar(x + 0.1, winter_df['Incident_Rate'], width, color='gray', label='Winter')
plt.bar(x + 0.3, spring_df['Incident_Rate'], width, color='green', label='Spring')

# Add labels and title
plt.xlabel('Province_State')
plt.ylabel('Incident Rate')
plt.title('COVID-19 Incident Rate by Province_State (All Seasons)')
plt.xticks(x, fall_df['Province_State'], rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# Access the dataframes
fall_df = dataframes_by_season['fall'][0]
summer_df = dataframes_by_season['summer'][0]
winter_df = dataframes_by_season['winter'][0]
spring_df = dataframes_by_season['spring'][0]

# Set up the figure
plt.figure(figsize=(12, 8))

# Define the positions of the bars
x = np.arange(len(fall_df['Province_State']))  # the label locations
width = 0.2  # the width of the bars

# Plotting data
plt.bar(x - 0.3, fall_df['Testing_Rate'], width, color='orange', label='Fall')
plt.bar(x - 0.1, summer_df['Testing_Rate'], width, color='blue', label='Summer')
plt.bar(x + 0.1, winter_df['Testing_Rate'], width, color='gray', label='Winter')
plt.bar(x + 0.3, spring_df['Testing_Rate'], width, color='green', label='Spring')

# Add labels and title
plt.xlabel('Province_State')
plt.ylabel('Testing Rate')
plt.title('COVID-19 Testing Rate by Province_State (All Seasons)')
plt.xticks(x, fall_df['Province_State'], rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
# import os
# import random


# # Define the seasons
# seasons = {
#     'jan': 'winter', 'feb': 'winter', 'mar': 'spring', 'apr': 'spring',
#     'may': 'spring', 'jun': 'summer', 'jul': 'summer', 'aug': 'summer',
#     'sep': 'fall', 'oct': 'fall', 'nov': 'fall', 'dec': 'winter'
# }

# # Set the directory path where the CSVs are stored
# directory = './datasets/csse_covid_19_daily_reports_us'

# # Create a dictionary to store selected CSVs by month
# selected_csvs_by_month = {month: [] for month in seasons.keys()}

# # List all files in the directory
# files = os.listdir(directory)

# # Iterate through files and group them by month
# for file in files:
#     for month in seasons.keys():
#         if month in file.lower() and file.endswith('.csv'):
#             selected_csvs_by_month[month].append(file)

# # Randomly select three CSVs from each month
# # selected_csvs = {month: random.sample(csvs, min(3, len(csvs))) for month, csvs in selected_csvs_by_month.items()}

# # Group selected CSVs by season
# seasonal_data = {season: [] for season in set(seasons.values())}

# for month, csvs in selected_csvs.items():
#     season = seasons[month]
#     seasonal_data[season].extend(csvs)

# # Print the grouped data
# for season, csvs in seasonal_data.items():
#     print(f'Season: {season}')
#     for csv in csvs:
#         print(f'  {csv}')

# # Create an empty dictionary to store dataframes by season
# dataframes_by_season = {season: [] for season in set(seasons.values())}

# # Load the selected CSVs into dataframes
# for season, csvs in selected_csvs_by_month.items():
#     for csv in csvs:
#         # Construct the file path
#         file_path = os.path.join(directory, csv)
        
#         # Load CSV into a dataframe
#         df = pd.read_csv(file_path)
        
#         # Append dataframe to the corresponding season
#         dataframes_by_season[season].append(df)

